<a href="https://colab.research.google.com/github/danicardonaibz/audio_classification/blob/main/ML_Material_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project in machine Learning for Materials Science

https://github.com/tonyreina/chemistry/blob/main/chemistry_predict_logP_tensorflow.ipynb

In [1]:
# First we will install some dependencies

%pip install --upgrade py3Dmol
%pip install --upgrade ase
%pip install --upgrade pubchempy
%pip install --upgrade dscribe
%pip install git+https://github.com/samoturk/mol2vec

%pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██                              | 1.9 MB 731 kB/s eta 0:00:38ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.7/http/client.py", line 465, in read
    n = self.readinto(b)
  File "/usr/lib/python3.7/http/client.py", line 509, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.7/ssl.py", line 1071, in recv_into
    return self.read(nbytes, buff

In [4]:
%pip install --upgrade rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.5 MB 80.3 MB/s 


In [5]:
import pandas as pd
import numpy as np
import re
from google.colab import drive
from itertools import zip_longest
from collections import defaultdict

import json
from matplotlib import pyplot as plt
from rdkit import Chem

## Step 1: Loading our data

In [6]:
drive.mount('/content/drive')

MessageError: ignored

In [7]:
df = pd.read_json("/content/drive/MyDrive/m1507656/df_62k.json", orient='split')

In [8]:
df.head(4)

,refcode_csd,canonical_smiles,inchi,number_of_atoms,xyz_pbe_relaxed,energies_occ_pbe,energies_occ_pbe0_vac_tier2,energies_occ_pbe0_water,energies_occ_pbe0_vac_tzvp,energies_occ_pbe0_vac_qzvp,...,energies_unocc_gw_qzvp,cbs_unocc_gw,total_energy_pbe,total_energy_pbe0_vac_tier2,total_energy_pbe0_water,total_energy_pbe0_vac_tzvp,total_energy_pbe0_vac_qzvp,hirshfeld_pbe,hirshfeld_pbe0_vac_tier2,hirshfeld_pbe0_water
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,InChI=1S/C18H18ClN3O3S2/c1-12-3-9-16(10-4-12)2...,45,45\n\nO 39.55724515 33.75271314 ...,"[-2759.71553, -2416.30896, -2412.55736, -513.9...","[-2788.05127, -2443.36662, -2439.20276, -526.4...",None,None,None,...,None,None,-63908.261677,-63911.485930,NaN,NaN,NaN,"[-0.27470711000000003, 0.44228032, -0.26425776...","[-0.29802765000000003, 0.47691104, -0.28757556...",None
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,InChI=1S/C28H22N2O2S/c29-21-26(27(22-13-5-1-6-...,55,55\n\nC 8.74281024 13.44863575 ...,"[-2416.32097, -510.97599, -510.93427, -382.106...","[-2443.36734, -523.36541, -523.31866, -392.998...",None,None,None,...,None,None,-47344.131203,-47346.537633,NaN,NaN,NaN,"[-0.04383635, 0.0489517, -0.04317942, 0.049465...","[-0.04511281, 0.05043578, -0.04467607, 0.05086...",None
2,LODZOT,n1ccc(cc1)c1nnc(o1)c1cccs1\t\n,InChI=1S/C11H7N3OS/c1-2-9(16-7-1)11-14-13-10(1...,23,23\n\nN 23.84904338 42.50577669 ...,"[-2412.75371, -513.88451, -382.11294, -382.041...","[-2439.38001, -526.39732, -393.09445, -393.002...",None,None,None,...,None,None,-28915.979909,-28916.826630,NaN,NaN,NaN,"[-0.15366001, 0.01755584, 0.05299325, -0.04774...","[-0.16233532, 0.02121239, 0.05594731, -0.06621...",None
3,LUSREW,CC/C=C(/S(=O)(=O)c1ccc(cc1)C)\F\t\n,"InChI=1S/C11H13FO2S/c1-3-4-11(12)15(13,14)10-7...",28,28\n\nC 36.52340453 39.64067030 ...,"[-2416.01387, -662.1882, -510.96036, -510.9032...","[-2443.06569, -676.06906, -523.38098, -523.310...","[-2443.54535, -676.3259, -523.94531, -523.9294...",None,None,...,None,None,-29310.525400,-29311.929019,-29312.152528,NaN,NaN,"[-0.10553088000000001, 0.052016810000000004, 0...","[-0.10641357, 0.053639570000000004, 0.05095191...","[-0.10396546, 0.06349725, 0.0567905, 0.0529875..."


In [9]:
df.reset_index(inplace=True, drop = True)

In [10]:
df.columns.tolist()

['refcode_csd',
 'canonical_smiles',
 'inchi',
 'number_of_atoms',
 'xyz_pbe_relaxed',
 'energies_occ_pbe',
 'energies_occ_pbe0_vac_tier2',
 'energies_occ_pbe0_water',
 'energies_occ_pbe0_vac_tzvp',
 'energies_occ_pbe0_vac_qzvp',
 'energies_occ_gw_tzvp',
 'energies_occ_gw_qzvp',
 'cbs_occ_gw',
 'energies_unocc_pbe',
 'energies_unocc_pbe0_vac_tier2',
 'energies_unocc_pbe0_water',
 'energies_unocc_pbe0_vac_tzvp',
 'energies_unocc_pbe0_vac_qzvp',
 'energies_unocc_gw_tzvp',
 'energies_unocc_gw_qzvp',
 'cbs_unocc_gw',
 'total_energy_pbe',
 'total_energy_pbe0_vac_tier2',
 'total_energy_pbe0_water',
 'total_energy_pbe0_vac_tzvp',
 'total_energy_pbe0_vac_qzvp',
 'hirshfeld_pbe',
 'hirshfeld_pbe0_vac_tier2',
 'hirshfeld_pbe0_water']

## Step 2: Preprocessing the data

### 2.1: Subsetting the dataframe

In [11]:
relevant_columns = ['refcode_csd', 'canonical_smiles', 'xyz_pbe_relaxed', 'total_energy_pbe']
df_subset = df[relevant_columns]
df_subset.head()

,refcode_csd,canonical_smiles,xyz_pbe_relaxed,total_energy_pbe
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,45\n\nO 39.55724515 33.75271314 ...,-63908.261677
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,55\n\nC 8.74281024 13.44863575 ...,-47344.131203
2,LODZOT,n1ccc(cc1)c1nnc(o1)c1cccs1\t\n,23\n\nN 23.84904338 42.50577669 ...,-28915.979909
3,LUSREW,CC/C=C(/S(=O)(=O)c1ccc(cc1)C)\F\t\n,28\n\nC 36.52340453 39.64067030 ...,-29310.525400
4,NOMBEA01,OC(=O)c1cc(N(=O)=O)c(c(c1)N(=O)=O)C\t\n,22\n\nC 3.95256786 17.67855833 24.82877401 \nN...,-23659.206412


### 2.2: Cleaning canonical SMILES
They happen to have \t\n characters to indicate end of line. These are going to be removed using a regex expression to match the rest of the strings.

In [12]:
df_subset["canonical_smiles"]

0        Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...
1        N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...
2                           n1ccc(cc1)c1nnc(o1)c1cccs1\t\n
3                      CC/C=C(/S(=O)(=O)c1ccc(cc1)C)\F\t\n
4                  OC(=O)c1cc(N(=O)=O)c(c(c1)N(=O)=O)C\t\n
                               ...                        
61484    COC(=O)c1c(c(C(=O)OC)c(c(c1C(C)(C)C)C(=O)OC)C(...
61485             O=C(c1ccccc1O)N/N=C/c1ccc(c(c1)Cl)Cl\t\n
61486         C1=CC=C2C[C]([CH]1)[C]1[CH]C=CC=C(C1)CC2\t\n
61487    Cc1ccc(cc1)S(=O)(=O)N[C@H](C(=O)N1C(=O)OC[C@@H...
61488    OC[C@H]1O[C@H]([C@@H]([C@@H]1O)O)c1n[nH]c2c1nc...
Name: canonical_smiles, Length: 61489, dtype: object

In [13]:
regex = "\\t\\n"
df_subset["canonical_smiles"]=df_subset["canonical_smiles"].apply(lambda x: re.sub(regex, "", x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_subset.head(2)

,refcode_csd,canonical_smiles,xyz_pbe_relaxed,total_energy_pbe
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,45\n\nO 39.55724515 33.75271314 ...,-63908.261677
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,55\n\nC 8.74281024 13.44863575 ...,-47344.131203


###2.2: Converting SMILES strings to RD-Kit molecules

In [15]:
df_subset["mol"] = df_subset["canonical_smiles"].apply(lambda x: (Chem.MolFromSmiles(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


RD-Kit outputs NoneType when fails to parse a SMILES string. Thus, to prevent exceptions when applying further transformations to the dataset, conflicting molecules are going to be removed.

In [16]:
df_subset_clean = df_subset[df_subset['mol'].apply(lambda x: x is not None)].reset_index(drop=True)
len(df_subset), len(df_subset_clean)

(61489, 61037)

In [17]:
# Adding hydrogen atoms to the molecules
df_subset_clean['mol'] = df_subset_clean['mol'].apply(lambda x: Chem.AddHs(x))

In [18]:
df_subset_clean.head(2)

,refcode_csd,canonical_smiles,xyz_pbe_relaxed,total_energy_pbe,mol
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,45\n\nO 39.55724515 33.75271314 ...,-63908.261677,<rdkit.Chem.rdchem.Mol object at 0x7f3f80426f30>
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,55\n\nC 8.74281024 13.44863575 ...,-47344.131203,<rdkit.Chem.rdchem.Mol object at 0x7f3f80427da0>


###2.3: Reshaping XYZ coordinates
TODO: apply MBTR to xyz to make a proper descriptor out of the atomic positions


In [19]:
df.iloc[0].xyz_pbe_relaxed

'45\n\nO      39.55724515      33.75271314      38.40917207 \nS      40.85906611      34.05164612      38.96498639 \nO      42.08066828      33.95284000      38.19595313 \nC      42.32571194      32.73322762      40.89834532 \nH      43.19968501      32.99032401      40.30295436 \nCl      45.20041227      29.36338131      46.43043646 \nC      41.05184934      33.09086636      40.45377016 \nC      42.45668317      32.04677947      42.10437138 \nH      43.45084814      31.76806349      42.45397500 \nC      41.33623713      31.70625411      42.87422993 \nC      40.06720668      32.07018496      42.39744704 \nH      39.18097405      31.80058002      42.97364913 \nC      39.91665528      32.75758494      41.19816166 \nH      38.92751728      33.01994634      40.82523353 \nC      41.48361320      30.98558291      44.18381422 \nH      42.41118162      30.40352859      44.22479339 \nH      40.63985814      30.30766109      44.36544785 \nH      41.51361125      31.70368231      45.01623603 \nC 

In [20]:
df_subset_clean.iloc[0].xyz_pbe_relaxed.split()[1:9]

['O',
 '39.55724515',
 '33.75271314',
 '38.40917207',
 'S',
 '40.85906611',
 '34.05164612',
 '38.96498639']

In [21]:
df_subset_clean.iloc[0].xyz_pbe_relaxed.split()[:10]

['45',
 'O',
 '39.55724515',
 '33.75271314',
 '38.40917207',
 'S',
 '40.85906611',
 '34.05164612',
 '38.96498639',
 'O']

In [22]:
def grouper(iterable, n, fillvalue=None):
  """
  This function splits the xyz array into N groups of 3 components, where N is
  the number of atoms in the molecule
  """
  args = [iter(iterable)] * n
  return list(zip_longest(*args, fillvalue=fillvalue))

In [23]:
def positions_to_dict(raw_xyz):
  """
  Takes in a string of xyz atom positions and returns a dict of atom positions
  """
  # We use defaultdict dict subclass because instead of raising a KeyError provides 
  # a default value for the key that does not exist. In this case, we want a list-like
  # object
  data_dict = defaultdict(list) # Specify what are the value types

  iter_array =  [iter(raw_xyz.split()[1:])] * 4 # Multiplied by 4 because [atom name, x, y z]
  grouped_xyz = list(zip_longest(*iter_array, fillvalue=None))

  for i, element in enumerate(grouped_xyz):
    data_dict[element[0]].append(list(element[1:]))
  
  return dict(data_dict)

In [24]:
df_subset_clean['xyz_pbe_relaxed'] = df_subset_clean['xyz_pbe_relaxed'].apply(positions_to_dict)

In [25]:
df_subset_clean['xyz_pbe_relaxed'].iloc[1]

{'C': [['8.74281024', '13.44863575', '23.16378372'],
  ['10.11074978', '13.19356923', '23.08694135'],
  ['10.78160584', '11.71087995', '26.56899447'],
  ['10.09912389', '12.35237176', '25.36230584'],
  ['8.72448552', '12.61600039', '25.43519330'],
  ['8.05098811', '13.15651134', '24.34266310'],
  ['10.78568512', '12.64534155', '24.18069782'],
  ['11.67094309', '6.71025632', '26.43329001'],
  ['10.61726558', '6.69117798', '25.51596520'],
  ['10.31077367', '7.84372826', '24.79244796'],
  ['11.03704148', '9.01628435', '24.99640142'],
  ['12.07051104', '9.77255700', '29.68626470'],
  ['13.29606260', '9.28321813', '30.13860887'],
  ['14.23360836', '10.14988375', '30.70242936'],
  ['13.95141000', '11.51436592', '30.81173061'],
  ['12.73005466', '12.01524457', '30.36397898'],
  ['11.79679134', '11.13595322', '29.80757459'],
  ['12.28663726', '11.49056361', '26.40928961'],
  ['13.05277476', '12.67860849', '26.58351282'],
  ['12.87960329', '10.27851286', '26.18676214'],
  ['12.08815605', '9.053

In [26]:
json.dumps(df_subset_clean['xyz_pbe_relaxed'].iloc[1], indent = 4)

'{\n    "C": [\n        [\n            "8.74281024",\n            "13.44863575",\n            "23.16378372"\n        ],\n        [\n            "10.11074978",\n            "13.19356923",\n            "23.08694135"\n        ],\n        [\n            "10.78160584",\n            "11.71087995",\n            "26.56899447"\n        ],\n        [\n            "10.09912389",\n            "12.35237176",\n            "25.36230584"\n        ],\n        [\n            "8.72448552",\n            "12.61600039",\n            "25.43519330"\n        ],\n        [\n            "8.05098811",\n            "13.15651134",\n            "24.34266310"\n        ],\n        [\n            "10.78568512",\n            "12.64534155",\n            "24.18069782"\n        ],\n        [\n            "11.67094309",\n            "6.71025632",\n            "26.43329001"\n        ],\n        [\n            "10.61726558",\n            "6.69117798",\n            "25.51596520"\n        ],\n        [\n            "10.31077367

In [27]:
df_subset_clean.head(2)

,refcode_csd,canonical_smiles,xyz_pbe_relaxed,total_energy_pbe,mol
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,"{'O': [['39.55724515', '33.75271314', '38.4091...",-63908.261677,<rdkit.Chem.rdchem.Mol object at 0x7f3f80426f30>
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,"{'C': [['8.74281024', '13.44863575', '23.16378...",-47344.131203,<rdkit.Chem.rdchem.Mol object at 0x7f3f80427da0>


### 2.4: Enriching the data with more features

In [28]:
from rdkit.Chem import Descriptors

df_subset_clean['tpsa'] = df_subset_clean['mol'].apply(lambda x: Descriptors.TPSA(x)) #https://en.wikipedia.org/wiki/Polar_surface_area
df_subset_clean['mol_w'] = df_subset_clean['mol'].apply(lambda x: Descriptors.ExactMolWt(x)) # https://en.wikipedia.org/wiki/Molecular_mass
df_subset_clean['num_valence_electrons'] = df_subset_clean['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x)) # https://en.wikipedia.org/wiki/Valence_electron
df_subset_clean['num_heteroatoms'] = df_subset_clean['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [29]:
df_subset_clean.head(2)

,refcode_csd,canonical_smiles,xyz_pbe_relaxed,total_energy_pbe,mol,tpsa,mol_w,num_valence_electrons,num_heteroatoms
0,ABAFEQ,Cc1ccc(cc1)S(=O)(=O)N[C@H](c1nnc(o1)SCc1ccc(cc...,"{'O': [['39.55724515', '33.75271314', '38.4091...",-63908.261677,<rdkit.Chem.rdchem.Mol object at 0x7f3f80426f30>,85.09,423.047811,142,9
1,ABEDOC,N#CC(=C(c1ccccc1)c1ccccc1)[C@H](c1ccccc1)NS(=O...,"{'C': [['8.74281024', '13.44863575', '23.16378...",-47344.131203,<rdkit.Chem.rdchem.Mol object at 0x7f3f80427da0>,69.96,450.140199,162,5


In [30]:
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D

import random

In [31]:
# Plotting some random molecules of the dataset
dopts = rdMolDraw2D.MolDrawOptions()
filename = 'mymolecules'

number_of_molecules = 6
to_draw = random.sample(range(0, len(df_subset_clean)), number_of_molecules)

mols = df_subset_clean["mol"][to_draw]
img = Draw.MolsToGridImage(mols, molsPerRow=3, useSVG=True,
                           legends=list(df["canonical_smiles"][to_draw].values),
                           drawOptions=dopts)

# Saving the file
if filename is not None:
    if not filename.endswith('.svg'):
        filename += '.svg'
    with open(filename, 'w') as f:
        f.write(img)

## Step 3: Deep Learning

### 3.1: Embedding SMILES for DL processing
TODO: Put this process in a preprocessing Keras layer

First, we're going to embed the molecules using a pre-trained model

In [32]:
import requests

url = 'https://github.com/samoturk/mol2vec_notebooks/raw/master/Notebooks/model_300dim.pkl'
file_to_download = requests.get(url, allow_redirects=True)

open('model_300dim.pkl', 'wb').write(file_to_download.content)

26567327

In [33]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec

In [34]:
#Loading pre-trained model via word2vec
from gensim.models import word2vec  # https://radimrehurek.com/gensim/index.html#install
w2vec_model = word2vec.Word2Vec.load('model_300dim.pkl') # Download from https://github.com/samoturk/mol2vec_notebo

In [35]:
#Constructing sentences
df_subset_clean['sentence'] = df_subset_clean.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

#Extracting embeddings to a numpy.array
#Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
df_subset_clean['mol2vec'] = [DfVec(x) for x in sentences2vec(df_subset_clean['sentence'], w2vec_model, unseen='UNK')]

In [36]:
df_subset_clean['mol2vec']=df_subset_clean['mol2vec'].apply(lambda x: x.vec)

In [37]:
df_subset_clean.columns.values.tolist()

['refcode_csd',
 'canonical_smiles',
 'xyz_pbe_relaxed',
 'total_energy_pbe',
 'mol',
 'tpsa',
 'mol_w',
 'num_valence_electrons',
 'num_heteroatoms',
 'sentence',
 'mol2vec']

In [38]:
final_dataset_columns=['xyz_pbe_relaxed',
 'tpsa',
 'mol_w',
 'num_valence_electrons',
 'num_heteroatoms',
 'mol2vec',
'total_energy_pbe']

final_dataset = df_subset_clean[final_dataset_columns]
final_dataset.head(2)

,xyz_pbe_relaxed,tpsa,mol_w,num_valence_electrons,num_heteroatoms,mol2vec,total_energy_pbe
0,"{'O': [['39.55724515', '33.75271314', '38.4091...",85.09,423.047811,142,9,"[-9.885212, 7.283, -11.909391, 2.714392, -7.30...",-63908.261677
1,"{'C': [['8.74281024', '13.44863575', '23.16378...",69.96,450.140199,162,5,"[-10.063227, 10.888024, -20.65452, 0.42244783,...",-47344.131203


# DFT calculations are expensive, so we would like to built a model capable of infering the molecule energy from their atom positions and species

## Converting molecules into ASE atoms

In [39]:
from ase import Atoms
from dscribe.descriptors import MBTR

In [40]:
# Step 1: Get all the atom values in the dataset
dict_positions = final_dataset.xyz_pbe_relaxed
species = []
for dictionary in dict_positions:
  for key, value in dictionary.items():
    if key not in species:
      species.append(key)

species[:3]

['O', 'S', 'C']

In [41]:
# Step 2: creating new column into the dataset with proper format for ase
def dict_to_tuples (x):
  """
  Takes in a dictionary of xyz and returns an array of tuples usable in Atoms.
  """
  positions = []
  names = []
  for key, value in x.items():
    for position in value:
      names.append(key)
      positions.append(tuple(map(lambda x: float(x), position)))
  return names, positions

In [42]:
final_dataset['xyz_pbe_relaxed'] = final_dataset['xyz_pbe_relaxed'].apply(dict_to_tuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
final_dataset.head(2)

,xyz_pbe_relaxed,tpsa,mol_w,num_valence_electrons,num_heteroatoms,mol2vec,total_energy_pbe
0,"([O, O, O, S, S, C, C, C, C, C, C, C, C, C, C,...",85.09,423.047811,142,9,"[-9.885212, 7.283, -11.909391, 2.714392, -7.30...",-63908.261677
1,"([C, C, C, C, C, C, C, C, C, C, C, C, C, C, C,...",69.96,450.140199,162,5,"[-10.063227, 10.888024, -20.65452, 0.42244783,...",-47344.131203


In [44]:
# Converting this last column into Atoms objects. This function cannot be parallelized
final_dataset['xyz_pbe_relaxed'] =  final_dataset['xyz_pbe_relaxed'].apply(lambda x: Atoms(x[0], x[1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
final_dataset.head(2)

,xyz_pbe_relaxed,tpsa,mol_w,num_valence_electrons,num_heteroatoms,mol2vec,total_energy_pbe
0,"(Atom('O', [39.55724515, 33.75271314, 38.40917...",85.09,423.047811,142,9,"[-9.885212, 7.283, -11.909391, 2.714392, -7.30...",-63908.261677
1,"(Atom('C', [8.74281024, 13.44863575, 23.163783...",69.96,450.140199,162,5,"[-10.063227, 10.888024, -20.65452, 0.42244783,...",-47344.131203


In [66]:
final_dataset.to_csv('gonzalo.csv')

## Creating MBTR descriptor


In [46]:
from dscribe.descriptors import MBTR

In [67]:
# Defining descriptor parameters
k1 = {
    "geometry": {"function": "atomic_number"},
    "grid": {"min": 1, "max": 85, "sigma": 0.1, "n": 20}  # Max atomic number is
                                                          # astatine
}

k2 = {
    "geometry": {"function": "inverse_distance"},
    "grid": {"min": 0.1, "max": 1, "sigma": 0.02, "n": 20}, # No distance smaller than 1 A
    "weighting": {"function": "exp", "r_cut": 10, "threshold": 1e-3}
}

k3 = {
    "geometry": {"function": "cosine"},
    "grid": {"min": -1, "max": 1, "sigma": 0.02, "n": 20},
    "weighting" : {"function": "exp", "r_cut": 10, "threshold": 1e-3}
}

In [68]:
# Instantiating the descriptor and determining number of features
mbtr = MBTR(species = species,
            periodic = False,
            k1 = k1,
            k2 = k2,
            k3 = k3,
            flatten = False,
            sparse = False,
            normalization = 'l2_each')
print(f"Number of features: {mbtr.get_number_of_features()}")

Number of features: 46560


In [51]:
subset = final_dataset.xyz_pbe_relaxed[1:10]

In [52]:
subset

1    (Atom('C', [8.74281024, 13.44863575, 23.163783...
2    (Atom('N', [23.84904338, 42.50577669, 35.03884...
3    (Atom('C', [36.52340453, 39.6406703, 25.169233...
4    (Atom('C', [3.95256786, 17.67855833, 24.828774...
5    (Atom('O', [20.69398953, 17.70752189, 21.42892...
6    (Atom('F', [23.77713885, 7.42382573, 20.976346...
7    (Atom('N', [23.32229332, 25.34462635, 30.41336...
8    (Atom('C', [20.83594705, 6.69063312, 19.653930...
9    (Atom('C', [33.20164464, 46.79185191, 50.18556...
Name: xyz_pbe_relaxed, dtype: object

In [55]:
demo_mbtr = mbtr.create(subset)

In [64]:
demo_mbtr[1]['k3']

array([[[[0.00122269, 0.00154   , 0.00085164, ..., 0.00108343,
          0.00379566, 0.00517995],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0. 

In [59]:
final_dataset.iloc[1].xyz_pbe_relaxed

Atoms(symbols='C28H22SO2N2', pbc=False)

In [61]:
28+22+1+2+2-22

33

In [ ]:
# final_dataset["xyz_pbe_relaxed"]=mbtr.create(final_dataset['xyz_pbe_relaxed'], n_jobs=2)

TerminatedWorkerError: ignored

In [ ]:
old_columns = final_dataset.columns.values.tolist()
old_columns

['xyz_pbe_relaxed',
 'tpsa',
 'mol_w',
 'num_valence_electrons',
 'num_heteroatoms',
 'mol2vec',
 'total_energy_pbe']

In [ ]:
# Create a dataset row by row and append values
columns = ['tpsa', 'mol_w', 'num_valence_electrons',
       'num_heteroatoms', 'mol2vec', 'total_energy_pbe', 'k1', 'k2', 'k3']
    
empty_dataset = pd.DataFrame(data=None, columns=columns)

for i in range(10):
    data = final_dataset.loc[i].to_dict()
    tensor_mbt = mbtr.create(data['xyz_pbe_relaxed'], n_jobs = 2)
    to_append = {
        'tpsa': data['tpsa'],
        'mol_w':data['mol_w'],
        'num_valence_electrons':data['num_valence_electrons'],
        'num_heteroatoms':data['num_heteroatoms'],
        'mol2vec':data['mol2vec'],
        'total_energy_pbe':data['total_energy_pbe'],
        'k1':np.array(tensor_mbt['k1']),
        'k2':np.array(tensor_mbt['k2']),
        'k3':np.array(tensor_mbt['k3'])
    }
    empty_dataset.loc[i] = to_append
empty_dataset
#empty_dataset.to_csv('test_short.csv', mode='a', header=False,index = False )
#print(f"Iteration number {i}.")

,tpsa,mol_w,num_valence_electrons,num_heteroatoms,mol2vec,total_energy_pbe,k1,k2,k3
0,85.09,423.047811,142,9,"[-9.885212, 7.283, -11.909391, 2.714392, -7.30...",-63908.261677,"[[0.69722646, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[[0.003923444, 0.009850678, 0.017902458, 0.02...",[[[[0.00072904 0.00191263 0.00093005 0.0021417...
1,69.96,450.140199,162,5,"[-10.063227, 10.888024, -20.65452, 0.42244783,...",-47344.131203,"[[0.6161233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0041563036, 0.0097708395, 0.016748, 0.023...",[[[[0.00080626 0.0014887 0.00111344 0.0007504...
2,51.81,229.030983,78,5,"[-5.338256, 3.8927343, -6.896216, -0.27919152,...",-28915.979909,"[[0.52689695, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[[0.0011403172, 0.0022627164, 0.0035649023, 0...",[[[[1.22269348e-03 1.54000171e-03 8.51641409e-...
3,34.14,228.062029,82,4,"[-4.85507, 5.27727, -9.770724, 2.1801903, -2.5...",-29310.525400,"[[0.7556151, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0030456602, 0.0062230816, 0.00918769, 0.0...",[[[[0.00021136 0.00077874 0.00118619 0.0036760...
4,123.58,226.022586,84,8,"[-3.7424397, 6.9630003, -7.5874896, 3.2785816,...",-23659.206412,"[[0.51075387, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[[0.0006998909, 0.0015201883, 0.0021145095, 0...",[[[[0.0000000e+00 0.0000000e+00 9.7732915e-09 ...
5,65.07,473.133065,170,8,"[-17.58828, 14.021827, -15.876698, -3.8287754,...",-58810.709608,"[[0.7391022, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.006383805, 0.01499319, 0.026573414, 0.038...",[[[[0.00161403 0.00322398 0.00404089 0.0041383...
6,74.60,237.988921,86,8,"[-2.0110483, -2.5596623, -2.713865, -0.4137087...",-27399.124352,"[[0.20006828, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[[0.00025718406, 0.00026242647, 0.00012195280...",[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7,80.31,218.966000,70,7,"[-6.113305, 0.741691, -2.0062485, -7.2529283, ...",-37742.357670,"[[0.55384904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[[7.787474e-05, 0.0006314617, 0.0023753194, 0...",[[[[0.00000000e+00 0.00000000e+00 0.00000000e+...
8,46.53,270.125594,104,3,"[-13.246321, 10.952683, -11.031231, -5.799448,...",-24062.403436,"[[0.7217342, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0017920027, 0.0068336683, 0.016360687, 0....",[[[[0.00028456 0.00051521 0.00092714 0.0011955...
9,150.27,333.096085,126,9,"[0.6289935, 9.114762, -8.443259, -13.040476, -...",-32555.343060,"[[0.6772855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0053187767, 0.008777954, 0.0091046905, 0....",[[[[0.00013888 0.00030406 0.00028701 0.0004968...


In [ ]:
empty_dataset.to_hdf('mycsvtest.hd5', format = 'table', key = 'hola')

TypeError: ignored

In [ ]:
import pathlib

In [ ]:
text = pathlib.Path('/content/mycsvtest.csv').read_text()
lines = text.split('\n')[1:-1]

all_strings = [float()]*9
all_strings

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
titanic_batches = tf.data.experimental.make_csv_dataset(
    '/content/mycsvtest.csv',
    label = '' batch_size=4)

In [ ]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived': {}".format(label_batch))
  print("features:")
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

ValueError: ignored

In [ ]:
tf.data.Dataset.from_tensor_slices(dict(empty_dataset))

ValueError: ignored

In [ ]:
import tensorflow as tf

In [ ]:
tf

In [ ]:
load_test = pd.read_csv('test_short.csv', names = columns)
np.array(load_test['k1'][1].split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


ValueError: ignored

In [ ]:
final_dataset.head(2).xyz_pbe_relaxed

,xyz_pbe_relaxed,tpsa,mol_w,num_valence_electrons,num_heteroatoms,mol2vec,total_energy_pbe
0,"(Atom('O', [39.55724515, 33.75271314, 38.40917...",85.09,423.047811,142,9,"[-9.885212, 7.283, -11.909391, 2.714392, -7.30...",-63908.261677
1,"(Atom('C', [8.74281024, 13.44863575, 23.163783...",69.96,450.140199,162,5,"[-10.063227, 10.888024, -20.65452, 0.42244783,...",-47344.131203


In [ ]:
final_dataset.mol2vec=final_dataset.mol2vec.to_numpy()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
final_dataset.mol2vec[1]

array([-10.063227  ,  10.888024  , -20.65452   ,   0.42244783,
        -6.409516  , -14.496673  ,  15.202631  ,  -4.9025784 ,
       -17.417852  , -40.73075   ,  35.362602  , -46.713898  ,
        17.616938  , -29.43649   , -13.078893  ,  22.052124  ,
       -13.164936  ,  23.158518  ,  -4.0208163 , -24.72941   ,
       -39.604656  ,  25.214321  , -13.413777  , -18.995308  ,
        17.376022  ,  27.263494  ,  20.827461  ,  -1.3330468 ,
        28.628298  ,   1.299943  ,  17.433578  ,  -5.419548  ,
        32.197598  ,  23.554342  ,  -8.341305  , -11.716682  ,
         2.4645536 ,   7.8396606 , -20.920656  ,  28.785887  ,
         3.9468193 , -47.194717  ,  34.89517   ,  -2.0419447 ,
       -13.47469   , -39.815014  , -25.715494  ,  44.340584  ,
        35.977043  ,  36.58457   , -20.07127   ,  -5.5532994 ,
       -39.61504   ,  13.610464  ,  21.153059  ,   1.9625216 ,
       -14.713921  , -44.166855  ,  21.553476  ,  27.542368  ,
        22.838907  , -16.055527  ,   6.0663323 ,  11.68

In [ ]:
# final_dataset.head()

In [ ]:
#test_dataframe_append
#for i in range(len(final_dataset)):
#    data = final_dataset.loc[[i]]
#    data.xyz_pbe_relaxed = data.xyz_pbe_relaxed.apply(lambda x: mbtr.create(x, n_jobs = 2))
#    data.to_csv('test_slices.csv',
#        index=False,
#        header=False,
#        mode='a')
#    print(f"Iteration number {i}.")

In [ ]:
#names = final_dataset.columns.values.tolist()


# Tensorflow analysis

import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.utils import plot_model

final_dataset.dtypes

test_tensor = mbtr.create(final_dataset.xyz_pbe_relaxed.iloc[1])

type(test_tensor['k3'])

def stack_dict(inputs, fun = tf.stack):
  values = []
  for key in sorted(inputs.keys()):
    values.append(tf.cast(inputs[key], tf.float32))
  return fun(values, axis = -1)

tf.cast(test_tensor['k3'], tf.float32)

inputs = {}
for name, column in test_tensor.items():
  inputs[name] = tf.keras.Input(
      shape=(1,), name=name, dtype=tf.float32)

inputs

test_tensor.items()

AttributeError: ignored